In [ ]:
#!pip install --upgrade cachetools==5.3.1

In [4]:
#!pip install streamlit

+ Compiling the code to work in visual code studio interfaz

In [5]:
import streamlit as st
import pandas as pd
import re
from googleapiclient.discovery import build
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics import jaccard_distance

# Load trend data file
file_path = 'C:/Users/Andrea/Documents/DATA/YT_Channel_Recommender/Database/df_trending.csv'
df_trending = pd.read_csv(file_path)
df_trending['category_id'] = df_trending['category_id'].astype('object')

# Connect my api credentialse
api_key = open("api_key_1.txt", "r").read()
youtube = build('youtube', 'v3', developerKey=api_key)

# Get the video ID for user input
def get_video_id(user_input):
    request = youtube.search().list(
        part="snippet",
        maxResults=1,
        q=user_input
    )
    response = request.execute()

    items = response['items']
    video_ids = []
    for item in items:
        if 'id' in item and 'videoId' in item['id']:
            video_id = item['id']['videoId']
        video_ids.append(video_id)

    if len(video_ids) > 0:
        return video_ids
    else:
        print("Sorry, I can't find your topic in my recommendations. Let's try another topic :).")
        return None

# Get video IDs
user_input = input("Type a topic for your channel: ")
video_ids = get_video_id(user_input)

# Get channel details
def get_tags_channel(video_ids):
    request = youtube.videos().list(
        part="snippet",
        id=video_ids
    )
    response = request.execute()

    database = []

    items = response['items']
    for item in items:
        video_data = [
            item['snippet'].get('channelId'),
            item['snippet'].get('channelTitle'),
            item['snippet'].get('categoryId'),
            item['snippet'].get('title'),
            item['snippet'].get('tags'),
            item['snippet'].get('description')
        ]

        channel_url = item['snippet'].get('channelId')
        request_1 = youtube.channels().list(
            part="snippet",
            id=channel_url
        )
        response_1 = request_1.execute()

        channel_data = response_1['items'][0]['snippet']
        video_data.append(channel_data.get('customUrl'))

        database.append(video_data)

    return database

# Get channel data
database = get_tags_channel(video_ids)

# Create DataFrame from channels
df_channel = pd.DataFrame(database, columns=['channel_id', 'channel_title', 'category_id', 'title', 'tags', 'description', 'custom_url'])

# Clean and process the tags
def clean_up(tag_list):
    cleaned_tags = []
    for tag in tag_list:
        tag = str(tag).lower()
        tag = re.sub("http:\S+", " ", tag)
        tag = re.findall("[a-z]+", tag)
        cleaned_tags.extend(tag)
    return ','.join(cleaned_tags)

def tokenize(tags):
    return word_tokenize(str(tags))

def lemmatize_input(tags):
    lem = WordNetLemmatizer()
    return [lem.lemmatize(word) for word in tags]

english_stopwords = stopwords.words('english')
spanish_stopwords = stopwords.words('spanish')
all_stopwords = english_stopwords + spanish_stopwords

def remove_sw_input(tags):
    return [word for word in tags if not word in all_stopwords]

def clean_tags(tags):
    tags = [tag for tag in tags if tag != ',']
    tags = list(set(tags))
    return tags

df_channel['tags'] = df_channel['tags'].apply(clean_up)
df_channel['tags'] = df_channel['tags'].apply(tokenize)
df_channel['tags'] = df_channel['tags'].apply(lemmatize_input)
df_channel['tags'] = df_channel['tags'].apply(remove_sw_input)
df_channel['tags'] = df_channel['tags'].apply(clean_tags)

# Filter the Trend DataFrame
def filter_trending_df(df_trending, category):
    filtered_df_trending = df_trending[df_trending['category_id'] == category].copy()
    
    if filtered_df_trending.shape[0] < 10:
        print("Sorry, we found just one channel. Please try again.")
        return None
    
    return filtered_df_trending

category = int(df_channel['category_id'].iloc[0])
filtered_df_trending = filter_trending_df(df_trending, category)

# NLP cleaning and compare the tags to calculate the score
def tokenizer_and_remove_punctuation(row):
    tokens = word_tokenize(row['tags'])
    return [word.lower() for word in tokens if word.isalnum()]

def lemmatize(row):
    lem = WordNetLemmatizer()
    return [lem.lemmatize(word) for word in row['tags']]

def remove_sw(row):
    return [word for word in row['tags'] if not word in all_stopwords]

def calculate_similarity(tags1, tags2):
    set_tags1 = set(tags1)
    set_tags2 = set(tags2)
    score = 1 - jaccard_distance(set_tags1, set_tags2)
    return score

def calculate_score(row):
    tags_channel = df_channel['tags'][0]
    tags_trending = row['tags']
    score = sum(calculate_similarity(tags_channel, tags_trending) for word_trending in tags_trending)
    return score

if filtered_df_trending is not None:
    filtered_df_trending['tags'] = filtered_df_trending.apply(tokenizer_and_remove_punctuation, axis=1)
    filtered_df_trending['tags'] = filtered_df_trending.apply(lemmatize, axis=1)
    filtered_df_trending['tags'] = filtered_df_trending.apply(remove_sw, axis=1)
    filtered_df_trending['tags'] = filtered_df_trending['tags'].apply(clean_tags)
    filtered_df_trending['score'] = filtered_df_trending.apply(calculate_score, axis=1)

    top_3_scores = filtered_df_trending.sort_values('score', ascending=False).head(3)

    def get_custom_url():
        channel_url = []
        for channel_id in top_3_scores['channel_id']:
            request = youtube.channels().list(
                part="snippet",
                id=channel_id
            )
            response = request.execute()
            items = response['items']
            for item in items:
                custom_url = item['snippet'].get('customUrl')
                channel_url.append(custom_url)
        return channel_url

    channel_url = get_custom_url()
    
top_3_scores['channel_url'] = channel_url
top_3_scores = top_3_scores.drop(columns=['channel_id', 'category_id', 'title', 'tags', 'description', 'score'])
top_3_scores


Type a topic for your channel: football boots


,channel_title,channel_url
508,CBS Sports Golazo,@cbssportsgolazo
558,Futbol y Curvas,@futbolycurvas
166,Selección Española de Fútbol (SeFutbol),@sefutbol


In [ ]:
# Display some video recommendation from the keywords
video_reccomendation = filtered_df_trending.sort_values('score', ascending=False).head(5).drop(columns=['channel_id', 'channel_title', 'category_id', 'tags', 'description','score'])
video_reccomendation = video_reccomendation.rename(columns={'title': 'Video recommendation / recomendación de video'})
video_reccomendation = video_reccomendation[['Video recommendation / recomendación de video']]
video_reccomendation

In [2]:
# Create the output as HTML
import pandas as pd
from html import escape

# Crea el documento HTML
doc = "<!DOCTYPE html>\n"
doc += "<html>\n"
doc += "<head>\n"
doc += "<style>\n"
doc += "table { border-collapse: collapse; width: 100%; }\n"
doc += "th, td { text-align: left; padding: 8px; }\n"
doc += "th { background-color: #f2f2f2; }\n"
doc += "tr:hover { background-color: #f5f5f5; }\n"
doc += "</style>\n"
doc += "</head>\n"
doc += "<body>\n"
doc += "<h1>YouTube Channel Recommender</h1>\n"
doc += "<p>Top 3 Recommended Channels:</p>\n"
doc += "<table>\n"
doc += "<thead>\n"
doc += "<tr>\n"
doc += "<th>Channel Title</th>\n"
doc += "<th>Custom URL</th>\n"
doc += "</tr>\n"
doc += "</thead>\n"
doc += "<tbody>\n"

for _, row in top_3_scores.iterrows():
    channel_title = escape(row['channel_title'])
    channel_url = escape(row['channel_url'])
    doc += "<tr>\n"
    doc += f"<td>{channel_title}</td>\n"
    doc += f"<td><a href='{channel_url}' target='_blank'>{channel_url}</a></td>\n"
    doc += "</tr>\n"

doc += "</tbody>\n"
doc += "</table>\n"
doc += "</body>\n"
doc += "</html>\n"

with open('output.html', 'w') as f:
    f.write(doc)
